In [1]:
from heeps.config.read_config import read_config
from heeps.config.update_config import update_config
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import remove_zernike
from heeps.util.img_processing import resize_cube
from astropy.io import fits
from copy import deepcopy
import numpy as np
import proper
import os

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
tag = 'Cfull_20211007'
scao = 'cfull/cube_%s'%tag + '_3600s_300ms_0piston_meters_%s_%s_%s.fits'
wv = 'wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV.fits'
pup = fits.getdata('wv/mask_Cbasic_20210601_720.fits')
pup[pup < .5] = 0
zpols = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100.fits')[::3]
n1 = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_1_nHF_1_G_0.4.fits')[::3]
n3 = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_3_nHF_3_G_0.4.fits')[::3]
n10 = fits.getdata('wv/cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_10_nHF_10_G_0.4.fits')[::3]
nimg = 720
temporal_rms = 8814.11
scaling_L = 100/temporal_rms
scaling_N = 1200/temporal_rms
print('scaling_L=%s, scaling_N=%s'%(np.round(scaling_L,3), np.round(scaling_N,3)))
npupil_L = update_config(**dict(read_config(), band='L'))['npupil']
npupil_N = update_config(**dict(read_config(), band='N2'))['npupil']
print('npupil_L=%s, npupil_N=%s'%(npupil_L, npupil_N))
wf = proper.prop_begin(1, 1, nimg, 1) # initial wavefront

scaling_L=0.011, scaling_N=0.136
npupil_L=285, npupil_N=119


## WV only

In [3]:
name = 'wv/cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_%s_%s_WVonly.fits'
try:
    wv_L = fits.getdata(name%('L', npupil_L))
    wv_N = fits.getdata(name%('N2', npupil_N))
    print('getdata ' + name)
except FileNotFoundError:
    print('writeto ' + name)
    wv_L = resize_cube(fits.getdata(wv)[::3], npupil_L)*scaling_L
    wv_N = resize_cube(fits.getdata(wv)[::3], npupil_N)*scaling_N
    fits.writeto(name%('L', npupil_L), wv_L)
    fits.writeto(name%('N2', npupil_N), wv_N)

getdata wv/cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_%s_%s_WVonly.fits


## SCAO + WV

In [4]:
name = 'cfull/wv/cube_%s'%tag + '_3600s_300ms_0piston_meters_%s_%s_%s_WV.fits'
if os.path.isfile(name%('all_ncpa', 'N2', npupil_N)):
    print('file exists: ' + name%('all_ncpa', 'N2', npupil_N))
else:
    for cube, band, nband in zip([wv_L, wv_N], ['L', 'N2'], [npupil_L, npupil_N]):
        for case in ['scao_only', 'all_ncpa']:
            print('writeto ' + name%(case, band, nband))
            fits.writeto(name%(case, band, nband), fits.getdata(scao%(case, band, nband)) + cube, overwrite=True)

file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV.fits


## SCAO + WV w/o tip-tilt

In [5]:
name = 'cfull/wv/cube_%s'%tag + '_3600s_300ms_0piston_meters_%s_%s_%s_WVnoTT.fits'
if os.path.isfile(name%('all_ncpa', 'N2', npupil_N)):
    print('file exists: ' + name%('all_ncpa', 'N2', npupil_N))
else:
    _, HSF = multiCPU(remove_zernike, case='noTT', multi_out=True, verbose=True,
        posargs=[deepcopy(wf), pup],
        posvars=[fits.getdata(wv)[::3], zpols[:,:3]])
    for scaling, band, nband in zip([scaling_L, scaling_N], ['L', 'N2'], [npupil_L, npupil_N]):
        wv_cube = resize_cube(HSF, nband)
        for case in ['scao_only', 'all_ncpa']:
            print('writeto ' + name%(case, band, nband))
            fits.writeto(name%(case, band, nband), fits.getdata(scao%(case, band, nband)) \
                + wv_cube*scaling, overwrite=True)

file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WVnoTT.fits


## QACITS

In [6]:
name = 'cfull/wv/cube_%s'%tag + '_3600s_300ms_0piston_meters_%s_%s_%s_WV_G_0.4_nLF_%s.fits'
for freq, zern in zip([1,3,10], [n1[:,:3], n3[:,:3], n10[:,:3]]):
    if os.path.isfile(name%('all_ncpa', 'N2', npupil_N, freq)):
        print('file exists: ' + name%('all_ncpa', 'N2', npupil_N, freq))
    else:
        _, HSF = multiCPU(remove_zernike, case='n%s'%freq, multi_out=True, verbose=True,
            posargs=[deepcopy(wf), pup],
            posvars=[fits.getdata(wv)[::3], zern])
        for scaling, band, nband in zip([scaling_L, scaling_N], ['L', 'N2'], [npupil_L, npupil_N]):
            wv_cube = resize_cube(HSF, nband)
            for case in ['scao_only', 'all_ncpa']:
                print('writeto ' + name%(case, band, nband, freq))
                fits.writeto(name%(case, band, nband, freq), fits.getdata(scao%(case, band, nband)) \
                    + wv_cube*scaling, overwrite=True)

file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_1.fits
file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_3.fits
file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_10.fits


## QACITS + PSI

In [7]:
name = 'cfull/wv/cube_%s'%tag + '_3600s_300ms_0piston_meters_%s_%s_%s_WV_G_0.4_nLF_1_nHF_%s.fits'
zern1 = np.hstack((n1[:,:3], n1[:,3:]))
zern3 = np.hstack((n1[:,:3], n3[:,3:]))
zern10 = np.hstack((n1[:,:3], n10[:,3:]))
for freq, zern in zip([1,3,10], [zern1, zern3, zern10]):
    if os.path.isfile(name%('all_ncpa', 'N2', npupil_N, freq)):
        print('file exists: ' + name%('all_ncpa', 'N2', npupil_N, freq))
    else:
        _, HSF = multiCPU(remove_zernike, case='n%s'%freq, multi_out=True, verbose=True,
            posargs=[deepcopy(wf), pup],
            posvars=[fits.getdata(wv)[::3], zern])
        for scaling, band, nband in zip([scaling_L, scaling_N], ['L', 'N2'], [npupil_L, npupil_N]):
            wv_cube = resize_cube(HSF, nband)
            for case in ['scao_only', 'all_ncpa']:
                print('writeto ' + name%(case, band, nband, freq))
                fits.writeto(name%(case, band, nband, freq), fits.getdata(scao%(case, band, nband)) \
                    + wv_cube*scaling, overwrite=True)

file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_1_nHF_1.fits
file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_1_nHF_3.fits
file exists: cfull/wv/cube_Cfull_20211007_3600s_300ms_0piston_meters_all_ncpa_N2_119_WV_G_0.4_nLF_1_nHF_10.fits
